In [379]:
import pandas as pd 
import numpy as np
import re
from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score

In [319]:
pd.options.display.max_colwidth = 280
pd.set_option('display.max_rows', 200)

In [320]:
data16 = pd.read_csv("covid19/2020-04-16.csv") 

In [ ]:
data17 = pd.read_csv("covid19/2020-04-17.csv") 
data18 = pd.read_csv("covid19/2020-04-18.csv") 
data19 = pd.read_csv("covid19/2020-04-19.csv") 
data20 = pd.read_csv("covid19/2020-04-20.csv") 
data21 = pd.read_csv("covid19/2020-04-21.csv") 
data22 = pd.read_csv("covid19/2020-04-22.csv") 
data23 = pd.read_csv("covid19/2020-04-23.csv") 
data24 = pd.read_csv("covid19/2020-04-24.csv") 
data25 = pd.read_csv("covid19/2020-04-25.csv") 
data26 = pd.read_csv("covid19/2020-04-26.csv") 
data27 = pd.read_csv("covid19/2020-04-27.csv") 
data28 = pd.read_csv("covid19/2020-04-28.csv") 
data29 = pd.read_csv("covid19/2020-04-29.csv") 
data30 = pd.read_csv("covid19/2020-04-30.csv") 

1.

In [321]:
data16 = data16[data16['lang']=='en']['text']

In [ ]:
data17 = data17[data17['lang']=='en']['text']
data18 = data18[data18['lang']=='en']['text']
data19 = data19[data19['lang']=='en']['text']
data20 = data20[data20['lang']=='en']['text']
data21 = data21[data21['lang']=='en']['text']
data22 = data22[data22['lang']=='en']['text']
data23 = data23[data23['lang']=='en']['text']
data24 = data24[data24['lang']=='en']['text']
data25 = data25[data25['lang']=='en']['text']
data26 = data26[data26['lang']=='en']['text']
data27 = data27[data27['lang']=='en']['text']
data28 = data28[data28['lang']=='en']['text']
data29 = data29[data29['lang']=='en']['text']
data30 = data30[data30['lang']=='en']['text']

In [322]:
df = pd.DataFrame(data=data16, columns=['text'])

In [323]:
df.head()

,text
5,G20 agrees to debt relief for poorest countries amid pandemic #COVID19 https://t.co/TDOwhg4Amd
7,B.C. health officials are urging people to kind and not jump to conclusions about travellers amid the #COVID19 pandemic. #Shuswap #kamloops https://t.co/GvIa7xXszv https://t.co/ZywH7k28d2
9,The Lovely Lisa Ann Joins The Show LIVE From NYC!! - @thereallisaann \n\n@RonDiazWDAE @IanBeckles @jayRecher \n\n#NFL #MLB #COVID19 #Coronavirus \n\nLISTEN \nhttps://t.co/2FF1qOng25
10,"@allisonnamias @ArriolaLR As @ethanbdm referenced, just saw this op-ed version on Ebola lessons for #COVID19 from @KateBaicker and Oeindrila Dube https://t.co/gLNq6CAzhW"
11,Study suggests higher rates of co-infection between #SARSCoV2 and other respiratory pathogens than previously reported https://t.co/oO3qd3RKnr #COVID19 #Coronavirus


In [324]:
df['alegria'] = df['text'].apply(lambda x: len(list(re.finditer(u'[\U0001F601-\U0001F606]|\U0001F609|\U0001F60A', str(x)))) > 0)

In [325]:
df['tristeza'] = df['text'].apply(lambda x: len(list(re.finditer(u'\U0001F613|\U0001F616|\U0001F621|\U0001F622|\U0001F62D|\U0001F630|\U0001F631|\U0001F64D|\U0001F64E', str(x)))) > 0)

In [326]:
df['enojo'] = df['text'].apply(lambda x: len(list(re.finditer(u'\U0001F620|\U0001F621|\U0001F624', str(x)))) > 0)

In [327]:
df['apoyo'] = df['text'].apply(lambda x: len(list(re.finditer(u'\U0001F637|\U0001F64B|\U0001F64C|\U0001F64F', str(x)))) > 0)

In [328]:
df = df[['text','alegria','tristeza','enojo','apoyo']]

In [329]:
df.head()

,text,alegria,tristeza,enojo,apoyo
5,G20 agrees to debt relief for poorest countries amid pandemic #COVID19 https://t.co/TDOwhg4Amd,False,False,False,False
7,B.C. health officials are urging people to kind and not jump to conclusions about travellers amid the #COVID19 pandemic. #Shuswap #kamloops https://t.co/GvIa7xXszv https://t.co/ZywH7k28d2,False,False,False,False
9,The Lovely Lisa Ann Joins The Show LIVE From NYC!! - @thereallisaann \n\n@RonDiazWDAE @IanBeckles @jayRecher \n\n#NFL #MLB #COVID19 #Coronavirus \n\nLISTEN \nhttps://t.co/2FF1qOng25,False,False,False,False
10,"@allisonnamias @ArriolaLR As @ethanbdm referenced, just saw this op-ed version on Ebola lessons for #COVID19 from @KateBaicker and Oeindrila Dube https://t.co/gLNq6CAzhW",False,False,False,False
11,Study suggests higher rates of co-infection between #SARSCoV2 and other respiratory pathogens than previously reported https://t.co/oO3qd3RKnr #COVID19 #Coronavirus,False,False,False,False


In [331]:
def data_preprocessing(source):
    source = source.lower()
    source = source.replace(',', ' ')
    source = source.replace('\n', ' ')
    source = source.replace('\r', '')
    source = re.sub(r'http\S+', '@URL', source)   #Links
    source = re.sub(r'\b\d\S+', '@NUM', source)   #Dates, Num
    source = source.replace('#coronavirus', '') 
    source = source.replace('#coronavirusoutbreak', '') 
    source = source.replace('#coronavirusPandemic', '') 
    source = source.replace('#covid19', '') 
    source = source.replace('#covid_19,,', '') 
    source = source.replace('#epitwitter', '') 
    source = source.replace('#ihavecorona', '') 
    source = source.replace('#StayHomeStaySafe', '') 
    source = source.replace('#TestTraceIsolate', '') 
    source = source.replace('\U0001F601', '') 
    source = source.replace('\U0001F602', '') 
    source = source.replace('\U0001F603', '') 
    source = source.replace('\U0001F604', '') 
    source = source.replace('\U0001F605', '') 
    source = source.replace('\U0001F606', '') 
    source = source.replace('\U0001F609', '') 
    source = source.replace('\U0001F60A', '') 
    source = source.replace('\U0001F616', '') 
    source = source.replace('\U0001F621', '') 
    source = source.replace('\U0001F622', '') 
    source = source.replace('\U0001F62D', '') 
    source = source.replace('\U0001F630', '') 
    source = source.replace('\U0001F631', '') 
    source = source.replace('\U0001F64D', '') 
    source = source.replace('\U0001F64E', '') 
    source = source.replace('\U0001F620', '') 
    source = source.replace('\U0001F621', '') 
    source = source.replace('\U0001F624', '') 
    source = source.replace('\U0001F637', '') 
    source = source.replace('\U0001F64B', '') 
    source = source.replace('\U0001F64C', '') 
    source = source.replace('\U0001F64F', '') 
    return source

data_train.loc[:,'text'] = data_train['text'].map(lambda x: data_preprocessing(x))

/Users/Aymeric/opt/anaconda3/envs/pandas_env/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [332]:
df.loc[:,'text'] = df['text'].map(lambda x: data_preprocessing(x))

In [346]:
def tokenize(x):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(x)

df['tokens'] = df['text'].map(tokenize)

In [357]:
df.head()

,text,alegria,tristeza,enojo,apoyo,tokens
5,g20 agrees to debt relief for poorest countries amid pandemic @url,False,False,False,False,"[g20, agrees, to, debt, relief, for, poorest, countries, amid, pandemic, url]"
7,b.c. health officials are urging people to kind and not jump to conclusions about travellers amid the pandemic. #shuswap #kamloops @url @url,False,False,False,False,"[b, c, health, officials, are, urging, people, to, kind, and, not, jump, to, conclusions, about, travellers, amid, the, pandemic, shuswap, kamloops, url, url]"
9,the lovely lisa ann joins the show live from nyc!! - @thereallisaann @rondiazwdae @ianbeckles @jayrecher #nfl #mlb listen @url,False,False,False,False,"[the, lovely, lisa, ann, joins, the, show, live, from, nyc, thereallisaann, rondiazwdae, ianbeckles, jayrecher, nfl, mlb, listen, url]"
10,@allisonnamias @arriolalr as @ethanbdm referenced just saw this op-ed version on ebola lessons for from @katebaicker and oeindrila dube @url,False,False,False,False,"[allisonnamias, arriolalr, as, ethanbdm, referenced, just, saw, this, op, ed, version, on, ebola, lessons, for, from, katebaicker, and, oeindrila, dube, url]"
11,study suggests higher rates of co-infection between #sarscov2 and other respiratory pathogens than previously reported @url,False,False,False,False,"[study, suggests, higher, rates, of, co, infection, between, sarscov2, and, other, respiratory, pathogens, than, previously, reported, url]"


In [348]:
df_train = df.loc[(df['alegria']>0) ^ (df['tristeza']>0) ^ (df['enojo']>0) ^ (df['apoyo']>0)]

In [284]:
df_train.to_csv('out.csv', encoding='utf-16', index=False)

In [336]:
def parse_values(x):
    if x['alegria']:
        return 0
    elif x['tristeza']:
        return 1
    elif x['enojo']:
        return 2
    elif x['apoyo']:
        return 3
    else:
        return None

In [349]:
df_train.loc[:,'category'] = df_train.apply(parse_values, axis=1)

/Users/Aymeric/opt/anaconda3/envs/pandas_env/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Aymeric/opt/anaconda3/envs/pandas_env/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [361]:
df.loc[:,'category'] = df.apply(parse_values, axis=1)

In [350]:
df_train.head()

,text,alegria,tristeza,enojo,apoyo,tokens,category
360,🕛@num @num 🌏2 @num @num 🏥1 @num @num 💀@num @num 👍@num @num 🇪🇺@num @num 🏥@num @num 💀@num @num 👍@num @num 🇺🇸@num @num 🏥@num @num 💀@num @num 👍@num @num 🇪🇸@num @num 🏥@num @num 💀@num @num 👍@num @num 🇮🇹@num @num 🏥@num @num 💀@num @num 👍@num @num total 🏥active 💀dead 👍recovered @url,False,False,False,True,"[num, num, 2, num, num, 1, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, num, total, active, dead, recovered, url]",3
460,robin sparkles: 🎶 don't go to the mall. at all. get the reference? #howimetyourmother,True,False,False,False,"[robin, sparkles, don, t, go, to, the, mall, at, all, get, the, reference, howimetyourmother]",0
713,""".....there are holes in the safety net in the shape of people.."" @shanekoyczan #google #animalcrossing 🏾🏾🏾 @url",False,False,False,True,"[there, are, holes, in, the, safety, net, in, the, shape, of, people, shanekoyczan, google, animalcrossing, url]",3
754,the washington national cathedral is lit up in blue every night this week to thank our healthcare workers 🩺🥼 #covid #healthcareworkers #thankyou @url,False,False,False,True,"[the, washington, national, cathedral, is, lit, up, in, blue, every, night, this, week, to, thank, our, healthcare, workers, covid, healthcareworkers, thankyou, url]",3
786,my heart ❤️ and prayers go out to everyone in the @wwe we're all in this together 👉👈💖 does not win we can and will overcome this 👉👈 #wwefamily #wweforever,False,False,False,True,"[my, heart, and, prayers, go, out, to, everyone, in, the, wwe, we, re, all, in, this, together, does, not, win, we, can, and, will, overcome, this, wwefamily, wweforever]",3


In [339]:
df_train.category.value_counts()

3    4750
0    3189
1    1118
2     139
Name: category, dtype: int64

In [390]:
X_train = df_train.text
y_train = df_train.category

In [365]:
X = df.text
y = df.category

In [356]:
pipe_mnnb = Pipeline(steps = [('tf', TfidfVectorizer()), ('mnnb', MultinomialNB())])

pgrid_mnnb = {
 'tf__max_features' : [1000, 2000, 3000],
 'tf__stop_words' : ['english', None],
 'tf__ngram_range' : [(1,1),(1,2)],
 'tf__use_idf' : [True, False],
 'mnnb__alpha' : [0.1, 0.5, 1]
}

gs_mnnb = GridSearchCV(pipe_mnnb,pgrid_mnnb,cv=5,n_jobs=-1)
gs_mnnb.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
    ...ue,
        vocabulary=None)), ('mnnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tf__max_features': [1000, 2000, 3000], 'tf__stop_words': ['english', None], 'tf__ngram_range': [(1, 1), (1, 2)], 'tf__use_idf': [True, False], 'mnnb__alpha': [0.1, 0.5, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [364]:
gs_mnnb.score(X_train, y_train)

0.72292301000434966

In [367]:
preds_mnnb = gs_mnnb.predict(X)
df['preds'] = preds_mnnb

In [369]:
df

,text,alegria,tristeza,enojo,apoyo,tokens,category,preds
5,g20 agrees to debt relief for poorest countries amid pandemic @url,False,False,False,False,"[g20, agrees, to, debt, relief, for, poorest, countries, amid, pandemic, url]",NaN,3
7,b.c. health officials are urging people to kind and not jump to conclusions about travellers amid the pandemic. #shuswap #kamloops @url @url,False,False,False,False,"[b, c, health, officials, are, urging, people, to, kind, and, not, jump, to, conclusions, about, travellers, amid, the, pandemic, shuswap, kamloops, url, url]",NaN,3
9,the lovely lisa ann joins the show live from nyc!! - @thereallisaann @rondiazwdae @ianbeckles @jayrecher #nfl #mlb listen @url,False,False,False,False,"[the, lovely, lisa, ann, joins, the, show, live, from, nyc, thereallisaann, rondiazwdae, ianbeckles, jayrecher, nfl, mlb, listen, url]",NaN,3
10,@allisonnamias @arriolalr as @ethanbdm referenced just saw this op-ed version on ebola lessons for from @katebaicker and oeindrila dube @url,False,False,False,False,"[allisonnamias, arriolalr, as, ethanbdm, referenced, just, saw, this, op, ed, version, on, ebola, lessons, for, from, katebaicker, and, oeindrila, dube, url]",NaN,0
11,study suggests higher rates of co-infection between #sarscov2 and other respiratory pathogens than previously reported @url,False,False,False,False,"[study, suggests, higher, rates, of, co, infection, between, sarscov2, and, other, respiratory, pathogens, than, previously, reported, url]",NaN,3
14,gabriel leung and colleagues from hong kong describe their actions after they were first alerted to a cluster of atypical pneumonia cases in wuhan. a figure shows the evolution of the epidemic and the responses to date in hong kong. #sarscov2,False,False,False,False,"[gabriel, leung, and, colleagues, from, hong, kong, describe, their, actions, after, they, were, first, alerted, to, a, cluster, of, atypical, pneumonia, cases, in, wuhan, a, figure, shows, the, evolution, of, the, epidemic, and, the, responses, to, date, in, hong, kong, sars...",NaN,3
15,the @ato_gov_au has provided smsf trustees with a course of action should a fund be in breach of the in-house assets rules due to covid-@num market downturns. read this article @url #smsf #trustees #financialplanning #smstrusteenews #covid__19,False,False,False,False,"[the, ato_gov_au, has, provided, smsf, trustees, with, a, course, of, action, should, a, fund, be, in, breach, of, the, in, house, assets, rules, due, to, covid, num, market, downturns, read, this, article, url, smsf, trustees, financialplanning, smstrusteenews, covid__19]",NaN,3
16,"""it's too late for my dad. my dad's gone and nothing's gonna bring him back."" workers at a meat plant in colorado have died of . across the u.s. and canada factory workers continue to work in elbow-to-elbow conditions. @url",False,False,False,False,"[it, s, too, late, for, my, dad, my, dad, s, gone, and, nothing, s, gonna, bring, him, back, workers, at, a, meat, plant, in, colorado, have, died, of, across, the, u, s, and, canada, factory, workers, continue, to, work, in, elbow, to, elbow, conditions, url]",NaN,3
17,"belarusian president lukashenka fresh out of a hockey game to a reporter: ""there are no viruses here. did you see any of them flying around?"" yet throughout april the number of officially confirmed cases has roughly doubled every three days. @url @url",False,False,False,False,"[belarusian, president, lukashenka, fresh, out, of, a, hockey, game, to, a, reporter, there, are, no, viruses, here, did, you, see, any, of, them, flying, around, yet, throughout, april, the, number, of, officially, confirmed, cases, has, roughly, doubled, every, three, days,...",NaN,0
20,we call on the senate to move forward on a vote for tps for venezuela now. resources should be spent on public health not on enforcement and detention. #venezuelatpsnow,False,False,False,False,"[we, call, on, the, senate, to, move, forward, on, a, vote, for, tps, for, venezuela, now, resourc

In [382]:
preds_train = gs_mnnb.predict(X_train)

In [397]:
y_train.as_matrix(columns=None)

array([3, 0, 3, ..., 0, 3, 1])

In [398]:
preds_train

array([3, 3, 3, ..., 0, 3, 0])

In [399]:
accuracy_score(y_train.as_matrix(columns=None), preds_train)

0.72292301000434966